<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/PromptDPSGD_softPrompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets opacus

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")

In [ ]:
from transformers import AutoTokenizer

checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
class Config:
  num_train_epochs = 3
  learning_rate = 5e-4
  n_prompt_tokens = 10
  random_range  = 0.5
  batch_size = 64
  max_grad_norm = 0.1
args = Config()

In [ ]:
from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["sentence"], max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


In [ ]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True, padding_value=0)
    labels = torch.stack([item['labels'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
from transformers import AutoModelForSequenceClassification


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class PROMPTEmbedding(nn.Module):
    def __init__(self,
                wte: nn.Embedding,
                n_tokens: int = 10,
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        super(PROMPTEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens,
                                                                               random_range,
                                                                               initialize_from_vocab))

    def initialize_embedding(self,
                             wte: nn.Embedding,
                             n_tokens: int = 10,
                             random_range: float = 0.5,
                             initialize_from_vocab: bool = True):
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(wte.weight.size(1), n_tokens).uniform_(-random_range, random_range)

    def forward(self, tokens):
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [ ]:
def get_new_soft_prompt_model(num_labels):
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)
  prompt_embedding = PROMPTEmbedding(model.get_input_embeddings(),
                      n_tokens=args.n_prompt_tokens,
                      initialize_from_vocab=True)
  model.set_input_embeddings(prompt_embedding)
  return model


In [ ]:
total_params = 0
for p in prompt_embedding.learned_embedding:
    total_params += p.numel()
print(total_params)

In [ ]:
total_params = 0
for p in model.parameters():
    total_params += p.numel()
print(total_params)

In [ ]:
def get_soft_prompt_optimiser(model):
  optimiser_grouped_parameters = [
      {
        "params": [p for n, p in model.named_parameters() if n == "bert.embeddings.word_embeddings.learned_embedding"],
      }
    ]

# Exclude other parameters from optimization
  for n, p in model.named_parameters():
      if not n == "bert.embeddings.word_embeddings.learned_embedding":
        p.requires_grad = False
      else:
       p.requires_grad =True

  optimiser = optim.SGD(optimiser_grouped_parameters, lr=args.learning_rate)
  return optimiser

In [ ]:
import numpy as np

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss, logits = outputs[:2]

        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        labels =batch['labels'].detach().cpu().numpy()

        loss_arr.append(loss.item())
        accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [ ]:
DELTA = 1 / len(train_dataloader)

In [ ]:
model = get_new_soft_prompt_model(2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model = model.train()

In [ ]:
LOGGING_INTERVAL = 100

In [ ]:
def train_soft_prompt(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions

            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)

                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                )

In [ ]:
def train_soft_prompt_DP(model,optimiser, train_dataloader):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    for  step, batch in enumerate(train_dataloader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions

            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )

In [ ]:
model =get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
train_soft_prompt(model, optimiser, train_dataloader)

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()
model = get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [ ]:
model =get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
train_soft_prompt_DP(model, optimiser, train_dataloader)

FOR QNLI dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_datasets = load_dataset("glue", "qnli")

In [ ]:
raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])

In [ ]:
from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question"],example["sentence"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence', 'question'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [ ]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
model = get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
train_soft_prompt(model, optimiser, train_dataloader)

In [ ]:
model =get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)


In [ ]:
from opacus import PrivacyEngine
model.train()
privacy_engine = PrivacyEngine()

model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [ ]:
train_soft_prompt_DP(model, optimiser, train_dataloader)

FOR QQP

In [ ]:
raw_datasets = load_dataset("glue", "qqp")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])
raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])
raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])

In [ ]:
from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question1"],example["question2"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'question1', 'question2'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [ ]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
model =get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
train_soft_prompt(model, optimiser, train_dataloader)

In [ ]:
model =get_new_soft_prompt_model(2)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [ ]:
train_soft_prompt_DP(model, optimiser, train_dataloader)

For MNLI

In [ ]:
raw_datasets = load_dataset("glue", "mnli")

In [ ]:
from datasets import load_metric
metric = load_metric("glue", "mnli")

In [ ]:

raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select([i for i in range(5000)])



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels =3)

In [ ]:
from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["premise"],example["hypothesis"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'premise', 'hypothesis'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [ ]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation_matched"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
model =get_new_soft_prompt_model(3)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
model.train()
train_soft_prompt(model, optimiser, train_dataloader)

In [ ]:
model =get_new_soft_prompt_model(3)
optimiser = get_soft_prompt_optimiser(model)

In [ ]:
from opacus import PrivacyEngine
model.train()
privacy_engine = PrivacyEngine()

model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,
)

In [ ]:
train_soft_prompt_DP(model, optimiser, train_dataloader)